## What's new

- generating .plt-files for visualization
- implementing a numerical flux
- definition and evaluation of the spatial operator
- explicit time integration

## Prerequisites

- projection onto a DG-field

Within this tutorial, we are going to implement the scalar transport equation via the definition of a spatial operator and an explicit time integrator. The implementation of the numerical flux is described on the basis of a upwinding scheme. For the visualization of the results, we are generating .plt-files, which can be opened by a viewer of your choice

## Problem statement
We are considering the following definition of the scalar transport equation with
$$

   \frac{\partial c}{\partial t} + \nabla \cdot (\vec{u} c) = 0,
$$
where $c = c(x,y,t) \in \mathbb{R}$ is the unknown concentration and
$(y,-x)$
is a given velocity field in $\Omega = [-1, 1] \times [-1, 1]$. Furthermore, the exact solution is given by
$$
    c_\text{Exact}(x,y,t) = \cos(\cos(t) x - \sin(t) y) \quad \text{ for } (x,y) \in \Omega
$$
In this tutorial we will simulate the evolution of the concentration $c(x,y,t)$.
# 1 Solution within the *BoSSS* framework
We start a new project

In [ ]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [ ]:
using System.IO;

Delete old plots in the current directory if any

In [ ]:
Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));

Error: (1,1): error CS0103: Der Name "Directory" ist im aktuellen Kontext nicht vorhanden.

# 2 Projection and visualization

In this first section we get to know the plotting tool **Tecplot**, 
which generates .plt-files of our **DGFields**. 
Previously, we define the exact solution $c_{Exact}(x,y,t)$ and 
the scalar components of the velocity field $\vec{u}$ as functions:

In [ ]:
public static class ExactSol {
    public static double c(double[] X, double t) => Math.Cos(Math.Cos(t)*X[0] - Math.Sin(t)*X[1]);
}

In [ ]:
public static class VelField {
    public static double u(double[] X) => X[1];
    public static double v(double[] X) => -X[0];
}

Next, we need to construct the computational domain, i.e a unit square with one cell.


In [ ]:
double[] nodes = GenericBlas.Linspace(-1.0, 1.0, 2); 
GridCommons grid = Grid2D.Cartesian2DGrid(nodes, nodes); 

We instantiate the **SinglePhaseField** *ch* with a **Basis** of DG-degreee of 2. Then we can project the initial value $c(x,y,0.0)$ onto *ch*.


In [ ]:
int dgDegree = 2;  
Basis basis = new Basis(grid, dgDegree);  
SinglePhaseField ch = new SinglePhaseField(basis, "ch");  
ch.ProjectField(X => ExactSol.c(X, 0.0));

Now, we can export the initial projection in our **Tecplot** format.

In [ ]:

using BoSSS.Solution.Tecplot;

Error: (2,7): error CS0246: Der Typ- oder Namespacename "BoSSS" wurde nicht gefunden (möglicherweise fehlt eine using-Direktive oder ein Assemblyverweis).

One important parameter for visualization is **superSampling**. It is essential for higher order methods since almost all
plotting tools work with piecewise linear interpolations of the data in the vertices. For our
case, the plot with **superSampling**=0 would just show a constant value! By increasing the
rate of the **superSampling**, we provide more sampling points for the plot tool.

- This has nothing to do with the computation! Only required for visualization!
- The number of sampling points grows exponentially with the value of
**superSampling**. Never use a value above 5 or 6!


In [ ]:
uint superSampling = 0;  
Tecplot tecplot    = new Tecplot(grid.GridData, superSampling);
//tecplot.PlotFields( 
//    "plot_tutorial4_superSampling0", 
//    0.0, 
//    ch);

In [ ]:
superSampling = 3;  
tecplot    = new Tecplot(grid.GridData, superSampling); 
//tecplot.PlotFields( 
//    "plot_tutorial4_superSampling3", 
//    0.0, 
//    ch);

There should now be two plot-files in your current directory. Those can be opened by any standard viewer for .plt-files.

# 3 Implementation of the (numerical) flux

Before we can define the spatial operator for the scalar transport equation, we need to implement the flux for the given problem, i.e. the flux of the divergence operator.

A flux defines the volume term 
(the **flux**) as well as the boundary terms (via the **numerical flux**).
We derive such a flux from the class **NonlinearFlux** which simplifies the implementation of fluxes in
flux-based formulations. 

In [ ]:

using BoSSS.Platform.LinAlg;  
class ScalarTransportFlux : NonlinearFlux {  
 
    /// <ArgumentOrdering> defines on which arguments this flux depends, and in which order. 
    /// Here, we have just one argument (the concentration <c>). The name is arbitrary at this point, but has to be 
    /// referenced when defining the spatial operator (see next section). Since this flux only depends on one
    /// argument, the parameters <U>, <Uin> and <Uout> will have a length of 1 and will contain the
    /// local values of $c$ in the first entry, i.e. <U[0]> in the method <Flux(...)>
    public override IList<string> ArgumentOrdering {  
        get { return new string[] { "c" }; }  
    }  
 
    ///<Flux(...)> defines the volume term. The array <output> (whose length is determined by the
    /// spatial dimension of the problem) has to contain the evaluated <flux> on exit.
    protected override void Flux(double time, double[] x, double[] U, double[] output) {  
        output[0] = VelField.u(x) * U[0];  
        output[1] = VelField.v(x) * U[0];  
    }  
 
    /// <InnerEdgeFlux(...)> defines the <numerical flux> between inner edges. The parameters 
    /// <Uin> and <Uout> contain the value from the <in> and <out> side, respectively, 
    /// where the normal vector <normal> points from <in> to <out>
    protected override double InnerEdgeFlux(double time, double[] x, double[] normal, 
    double[] Uin, double[] Uout, int jEdge) {  
        Vector n              = new Vector(normal);  
        Vector velocityVector = new Vector( VelField.u(x),  VelField.v(x));  
 
        if (velocityVector * n > 0) {  
            return (velocityVector * Uin[0]) * n;  
        } else {  
            return (velocityVector * Uout[0]) * n;  
        }  
    }  
 
    /// <BorderEdgeFlux(...)> defines the <numerical flux> at boundary edges, where only inner values 
    /// (<Uin>) are given. Here, we reuse <InnerEdgeFlux(...)> and the exact solution <cExact> 
    /// to define a suitable boundary condition.
    protected override double BorderEdgeFlux(double time, double[] x, double[] normal, 
                                             byte EdgeTag, double[] Uin, int jEdge) {  
        double[] Uout = new double[] { ExactSol.c(x, time) };  
        return InnerEdgeFlux(time, x, normal, Uin, Uout, jEdge);  
    }  
} 
/// \leftskip=0cm

# 4 Definition of a spatial operator

The next step is the definition of the spatial operator.

In [ ]:

var spatialTerm = new DifferentialOperator( 
    new string[] { "c" },        // Domain variable 
    new string[] { "div" },      // Co-domain variable
    QuadOrderFunc.NonLinear(2)); // Order of integration


The name of the **domain variable** must be the same used in **ArgumentOrdering**
in the definition of the flux, i.e **ScalarTransportFlux**.
The name of the **co-domain variable** is arbitrary and is used when the 
fluxes are added. In our case, we only have one type of flux.


**QuadOrderFunc.NonLinear(int x)** computes the required integration order 
for a non-linear flux. Here, the flux is given by $\vec{u} c$, where $\vec{u}$ is linear. 

So, we have second order terms (flux times the ansatz functions), i.e the required
order is **2*dgDegree+1** 

We add the flux of the divergence operator to the corresponding equation component, identified by the 
**co-domain variable**,

In [ ]:

spatialTerm.EquationComponents["div"].Add(new ScalarTransportFlux());

and finalize the definition of the operator

In [ ]:
spatialTerm.Commit();

# 5 Time integration

Finally, we have to specify a time stepping scheme to solve the time dependent
scalar transport equation. For simplification, we use 
the **ExplicitEuler** scheme, which just needs the **DifferentialOperator** and 
the **DGField** as arguments.

In [ ]:

using BoSSS.Solution.Timestepping;

In [ ]:
ExplicitEuler timeStepper = new ExplicitEuler(spatialTerm, ch);

We want to perform a full revolution ($t \in [0, 2\pi]$) with 24 timesteps.

In [ ]:
double endTime = 2.0 * Math.PI; 
int numberOfTimesteps = 24;

Plot the initial data

In [ ]:
tecplot.PlotFields( 
    "plot_tutorial4_0", 
    0.0, 
    ch); 

Now, we can start the simulation, where the **timestepper** performs in each iteration one
explicit euler timestep with the timestep size **dt**

In [ ]:
double dt = endTime / numberOfTimesteps; 
for (int i = 1; i <= numberOfTimesteps; i++) { 
    timeStepper.Perform(dt); 
    tecplot.PlotFields(          // plot each timestep
        "plot_tutorial4_" + i, 
        timeStepper.Time, 
        ch); 
}

Error: (1,13): error CS0103: Der Name "endTime" ist im aktuellen Kontext nicht vorhanden.
(1,23): error CS0103: Der Name "numberOfTimesteps" ist im aktuellen Kontext nicht vorhanden.
(2,22): error CS0103: Der Name "numberOfTimesteps" ist im aktuellen Kontext nicht vorhanden.
(3,5): error CS0103: Der Name "timeStepper" ist im aktuellen Kontext nicht vorhanden.
(4,5): error CS0103: Der Name "tecplot" ist im aktuellen Kontext nicht vorhanden.
(6,9): error CS0103: Der Name "timeStepper" ist im aktuellen Kontext nicht vorhanden.
(7,9): error CS0103: Der Name "ch" ist im aktuellen Kontext nicht vorhanden.

Furthermore, we can postprocess our simulation data in various ways: For example, 
we can compute the L2-Error at the end of the simulation

In [ ]:
double error = ch.L2Error(X => ExactSol.c(X, timeStepper.Time)); 
error

0.10126945730309017

# 6 Advanced topics 
So far we used the **timestepper** to evaluate the **DifferentialOperator** in order to get 
the current change rate for the explicit Euler scheme. In the next section we will evaluate the operator 
in each iteration manually. But first we set the **DGField** back to the initial values and plot


In [ ]:
ch.ProjectField(X => ExactSol.c(X, 0.0));
tecplot.PlotFields( 
    "plot_tutorial4_advanced_0", 
    0.0, 
    ch);


## Evaluation of the spatial operator
To evaluate the **DifferentialOperator**, we have to provide a mapping of the DG-coordinates of **ch**.
This describes a bijective mapping between **local unique indices** and **global unique indices**


In [ ]:
var mapping = new CoordinateMapping(ch);

In other words, it maps the DG-coordinates into one long, one-dimensional **CoordinateVector**


In [ ]:
var DGCoordinates = new CoordinateVector(mapping);

Now, we can create an evaluator of the **DifferentialOperator** for the **DGField** **ch**

In [ ]:

var evaluator = spatialTerm.GetEvaluatorEx(mapping.Fields, null, mapping);

In our case this evaluator computes the fluxes of the divergence operator evaluated for the **DGField** **ch**


In [ ]:
double[] flux = new double[ch.CoordinateVector.Count]; 
evaluator.time = 0.0;
evaluator.Evaluate(1.0, 0.0, flux);


After the evaluation the output flux is 
>flux = 0.0 $\cdot$ flux + 1.0 $\cdot$ spatialTerm(time: 0.0)

Finally, we can use this kind of **spatial operator** evaluation to implement the explicit Euler scheme manually.   


In [ ]:
double physTime = 0.0;
for (int i = 1; i <= numberOfTimesteps; i++) { 
    evaluator.time = physTime;
    evaluator.Evaluate(1.0, 0.0, flux);
    DGCoordinates.axpy<double[]>(flux, -dt);
    physTime += dt;
    tecplot.PlotFields(        
        "plot_tutorial4_advanced" + i, 
        physTime, 
        ch); 
}

In [ ]:
double error = ch.L2Error(X => ExactSol.c(X, physTime)); 
error

0.10126945730309017